In [ ]:
%load_ext autoreload
%autoreload 2

## sample model db

In [ ]:
from src.openai.requests import input_description, output_description
import pandas as pd

In [ ]:
data = pd.read_json(path_or_buf="data/huggingface_models.jsonl", lines=True)
model_db = data[["id","task","description"]].sample(10)

In [ ]:
model_db["input_desc"] = model_db.description.apply(input_description)

In [ ]:
model_db["output_desc"] = model_db.description.apply(output_description)

In [ ]:
# save to json
model_db.to_json("data/model_db.json", orient="records")

## task planning

In [ ]:
from src.openai.requests import zero_shot_task_planning

In [ ]:
zero_task_plan = zero_shot_task_planning("a model that takes as input text from user change it to a poem and classify it as positive emotion or negative emotion")

In [ ]:
final = eval(zero_task_plan)

In [ ]:
final

## input modeling

In [ ]:
test = """
we need to make it easy for our employees to add and edit regulations. 
Adding a new regulation requires mutiple checks across thousands of pages of already existing regulations. 
During these checks (which takes months) emplyees look for conflicting regulations made earlier, 
loop holes and blind spots.
find the relevent regulations, paragraphs and articles to the request
check if a conflict occurs
explain the conflict occuring
"""

In [ ]:
test_task_plan= zero_shot_task_planning(test)

In [ ]:
final = eval(test_task_plan)
final

In [ ]:
# subject:str, task:str, description:str, inputs:dict, outputs:dict
def problem(**kwargs):
    return str(kwargs)

In [ ]:
problem_1 = problem(subject="document embedding", 
        task="text splitting", 
        description="divide text into chunks", 
        inputs="[document text : str]", 
        outputs="[list of sentences : list[str]]")

problem_2 = problem(subject="document embedding", 
        task="text splitting", 
        description="divide text into chunks", 
        inputs="[document text : str]", 
        outputs="[list of sentences : list[str]]")

In [ ]:
problems = [
"""
{"subject" : document embedding
task : text splitting
description : divide text into chunks
inputs : [document text : str]
output : list of sentences : list[str]}
""",
"""
"subject" : document embedding
task : pdf to text
description : turn the pdf into text while ignoring images
inputs : [input documentation pdf : pdf]
output : document text : str
""",
"""
"subject" : document embedding
task : sentence embedding
description : embed each sentence
inputs : [list of sentences : list[str]]
output : list of list of embeddings and list of sentences : list[list[str], list[float]]] 
"""]


## model selection

In [ ]:
from src.model_selection.model_selection import SemanticSearchEngine
from src.examples.test_task_plans import test_task_plan_1, test_task_plan_2, test_task_plan_3
from src.utils.utils import read_json

In [ ]:
model_db = read_json("data/model_db.json")

In [ ]:
se = SemanticSearchEngine()
se.forward(model_db=model_db, task_db=test_task_plan_1)